# CSS @supports Analysis

In [53]:
import os
import json
import pprint

In [54]:
# https://github.com/mdn/browser-compat-data/blob/main/schemas/compat-data-schema.md

browser_to_release_to_features = dict()
last_n_browsers = 10
skips = 0
flag_features = set()

filepath = 'browser-compat-data.json'
with open(filepath, 'r') as file:
    data = json.load(file)
    
    browsers = set(data['browsers'].keys())
    browsers.remove('deno')
    browsers.remove('nodejs')
    browsers.remove('oculus')
    browsers = ['chrome', 'firefox', 'safari']
    pprint.pprint(browsers)

    num_releases = 0
    # setup releases in correct order
    browser_to_release_order = dict()
    for browser in browsers:
        with open(f"browser-compat-data-browsers/{browser}.json", 'r') as browser_file:
            browser_data = json.load(browser_file)
            browser_data = browser_data['browsers'][browser]
        releases = list(browser_data['releases'].keys())[-last_n_browsers:]
        num_releases += len(releases)

        browser_to_release_to_features[browser] = { v: set() for v in releases }
        browser_to_release_order[browser] = list(releases)
    
    print(f"Number of Releases: {num_releases}")
    # iterate properties and update all releases that support a prop
    features = [('properties', x) for x in data['css']['properties'].keys()] + [('at-rules', x) for x in data['css']['at-rules'].keys()]  + [('selectors', x) for x in data['css']['selectors'].keys()] + [('types', x) for x in data['css']['types'].keys()]
    for category, prop in features:
        all_supports = [(prop, data['css'][category][prop]['__compat']['support'])]
        for sub_prop in data['css'][category][prop]:
            if sub_prop == "__compat":
                continue
            all_supports.append((sub_prop, data['css'][category][prop][sub_prop]['__compat']['support']))

        for clear_name, support in all_supports:
            for browser in browsers:
                browser_support_ranges = support[browser]
                if not isinstance(browser_support_ranges, list):
                    browser_support_ranges = [browser_support_ranges]
                
                for browser_support in browser_support_ranges:
                    if 'prefix' in browser_support:
                        clear_name = browser_support['prefix'] + prop
                    if 'alternative_name' in browser_support:
                        clear_name = browser_support['alternative_name']

                    if browser_support['version_added'] == False:
                        continue
                    if browser_support['version_added'] == 'preview':
                        skips += 1
                        continue
                    if browser_support['version_added'] == True:
                        # TODO: what should we do in this case? (i.e., unknown version)
                        skips += 1
                        continue
                    if "flags" in browser_support:
                        # TODO: feature enabled or disabled by flag
                        flag_features.add(clear_name)
                        skips += 1
                        continue
                    start = browser_support['version_added']
                    start = start.replace("≤", "")
                    
                    end = None
                    if 'release_removed' in browser_support and browser_support['release_removed'] != False:
                        end = browser_support['release_removed']
                        end = end.replace("≤", "")

                    if start not in browser_to_release_to_features[browser]:
                        continue

                    # add to all supported versions
                    supported = False
                    for release in browser_to_release_to_features[browser]:
                        if release == start:
                            supported = True
                        if release == end:
                            supported == False
                        if supported:
                            browser_to_release_to_features[browser][release].add(clear_name)

['chrome', 'firefox', 'safari']
Number of Releases: 30


In [55]:
unique_set_to_releases = dict()

# generate clusters
for browser in browser_to_release_to_features:
    for release in browser_to_release_to_features[browser]:
        props = frozenset(browser_to_release_to_features[browser][release])
        if props not in unique_set_to_releases:
            unique_set_to_releases[props] = set()
        unique_set_to_releases[props].add(f"{browser} {release}")

In [56]:
clusters = set()
for unique in unique_set_to_releases:
    clusters.add(tuple(unique_set_to_releases[unique]))

pretty_clusters = sorted(sorted(list(map(lambda x: sorted(x), clusters)), key=lambda x: len(x), reverse=True), key=lambda x: x[0])

### All Clusters and Differences between them

In [57]:
pprint.pprint(pretty_clusters)

[['chrome 113'],
 ['chrome 114'],
 ['chrome 115'],
 ['chrome 116'],
 ['chrome 117'],
 ['chrome 118'],
 ['chrome 119'],
 ['chrome 120'],
 ['chrome 121', 'chrome 122'],
 ['firefox 115'],
 ['firefox 116'],
 ['firefox 117'],
 ['firefox 118'],
 ['firefox 119'],
 ['firefox 120'],
 ['firefox 121', 'firefox 122', 'firefox 123', 'firefox 124'],
 ['safari 16.1'],
 ['safari 16.2', 'safari 16.3'],
 ['safari 16.4'],
 ['safari 16.5', 'safari 16.6'],
 ['safari 17', 'safari 17.1'],
 ['safari 17.2', 'safari 17.3']]


In [58]:
previous_cluster = pretty_clusters[0]
browser, release = previous_cluster[0].split(" ")
previous_features = set(browser_to_release_to_features[browser][release])
for cluster in pretty_clusters[1:]:
    browser, release = cluster[0].split(" ")
    features = set(browser_to_release_to_features[browser][release])
    additions = features - previous_features
    removals = previous_features - features

    print(f"\n{previous_cluster} -> {cluster}")
    print("+ " + str(additions))
    print("- " + (str(removals) if len(removals) > 0 else "{}"))

    # update
    previous_cluster = cluster
    previous_features = features



['chrome 113'] -> ['chrome 114']
+ {'popover-open', 'closed', 'shorthand_values', 'wrap', 'white-space-collapse', 'open', 'nowrap', 'text-wrap', 'popover', 'balance'}
- {}

['chrome 114'] -> ['chrome 115']
+ {'animation-range-end', 'animation-timeline', 'view-timeline-inset', 'scroll-timeline-name', 'animation-range-start', 'scroll-timeline-axis', 'view-timeline-name', 'view-timeline', 'scroll', 'auto', 'scroll-timeline', 'multi-keyword_values', 'animation-range', 'view', 'named_range_keyframes', 'view-timeline-axis'}
- {}

['chrome 115'] -> ['chrome 116']
+ {'offset-position', 'size', 'timeline-scope', 'basic-shape', 'offset-anchor', 'coord-box', 'normal', 'rect', 'xywh', 'url', 'keyframe_animatable', 'position', 'ray'}
- {}

['chrome 116'] -> ['chrome 117']
+ {'subgrid', 'font-variant-position', 'auto_none', 'transition_behavior_value', 'cap', 'transitionable', 'rcap', 'pretty', 'overlay', 'transition-behavior', 'starting-style'}
- {}

['chrome 117'] -> ['chrome 118']
+ {'scope', 's

### Unique Browser Releases

In [59]:
print(f"{len(list(filter(lambda x: len(x) == 1, clusters)))} / {sum(map(lambda x: len(x), clusters))} = {len(list(filter(lambda x: len(x) == 1, clusters))) / sum(map(lambda x: len(x), clusters))}")
pprint.pprint(list(filter(lambda x: len(x) == 1, clusters)))
pprint.pprint(list(filter(lambda x: len(x) != 1, clusters)))

16 / 30 = 0.5333333333333333
[('firefox 120',),
 ('chrome 115',),
 ('chrome 120',),
 ('chrome 118',),
 ('safari 16.4',),
 ('chrome 119',),
 ('firefox 117',),
 ('firefox 115',),
 ('chrome 117',),
 ('safari 16.1',),
 ('chrome 116',),
 ('chrome 113',),
 ('chrome 114',),
 ('firefox 118',),
 ('firefox 119',),
 ('firefox 116',)]
[('safari 17.3', 'safari 17.2'),
 ('chrome 121', 'chrome 122'),
 ('safari 17.1', 'safari 17'),
 ('safari 16.5', 'safari 16.6'),
 ('firefox 123', 'firefox 124', 'firefox 121', 'firefox 122'),
 ('safari 16.2', 'safari 16.3')]


### Diff of two releases

In [60]:
repr_a, repr_b = "chrome 121", "firefox 122" # latest on windows 11
browser_a, release_a = repr_a.split(" ")
browser_b, release_b = repr_b.split(" ")

features_a = set(browser_to_release_to_features[browser_a][release_a])
features_b = set(browser_to_release_to_features[browser_b][release_b])
additions = features_a - features_b
removals = features_b - features_a

print(f"\n{repr_a} -> {repr_b}")
print("+ " + (str(additions) if len(additions) > 0 else "{}"))
print("- " + (str(removals) if len(removals) > 0 else "{}"))
print(len(additions) + len(removals))



chrome 121 -> firefox 122
+ {'rcap', 'animation-timeline', 'shorthand_values', 'scroll-timeline-axis', 'view-timeline-name', 'content-box', 'transition_behavior_value', 'animation-range-start', 'wrap', 'linear-function', 'scrollbar-width', 'view-timeline', 'spelling-error', 'offset-position', 'grammar-error', 'font-variant-position', 'white-space-collapse', 'overflow-inline', 'fill_and_stroke_box', 'view-timeline-axis', 'popover-open', 'mask-size', 'rect', 'mask-mode', 'offset-anchor', 'prefers-reduced-transparency', 'user-valid', 'dir', 'open', 'auto-phrase', 'scroll', 'update', 'normal', 'scroll-timeline-name', 'mask-image', 'popover', 'keyframe_animatable', 'ray', 'subgrid', 'scope', 'fill-box', 'mask-repeat', 'scripting', 'flow_relative_values', 'cap', 'scrollbar-color', 'scroll-timeline', 'image-set', 'transitionable', 'coord-box', 'overlay', 'multi-keyword_values', 'animation-range', 'border-box', 'size', 'closed', 'flow_relative_support', 'pretty', 'auto', 'xywh', 'url', 'overf

### Meta

In [61]:
print(f"Number of Skips: {skips}")

Number of Skips: 84


In [62]:
print(len(flag_features))
pprint.pprint(flag_features)

39
{'-moz-submit-invalid',
 'OpenType_COLRv1',
 'align-tracks',
 'animation-timeline',
 'basic-shape',
 'color-contrast',
 'content-visibility',
 'coord-box',
 'fit-content_function',
 'font-variant-emoji',
 'inverted-colors',
 'justify-tracks',
 'line-height-step',
 'masonry',
 'none_applies_to_elements',
 'normal',
 'number_value',
 'offset-position',
 'popover',
 'popover-open',
 'position',
 'prefers-reduced-data',
 'prefers-reduced-transparency',
 'ray',
 'rect',
 'scroll',
 'scroll-timeline',
 'scroll-timeline-axis',
 'scroll-timeline-name',
 'scrollbar-gutter',
 'size',
 'text-justify',
 'url',
 'video-dynamic-range',
 'view',
 'view-timeline',
 'view-timeline-axis',
 'view-timeline-name',
 'xywh'}
